# Spark DataFrames

Last part of https://towardsdatascience.com/the-hitchhikers-guide-to-handle-big-data-using-spark-90b9be0fe89a

In [2]:
# File 'data' is a CSV that uses TAB as separator and does not have header
#ratings = spark.read.load("Data-ML-100k--master/ml-100k/u.data",format="csv", sep="\t", inferSchema="true", header="false")
ratings = spark.read.load("/FileStore/tables/u.data",format="csv", sep="\t", inferSchema="true", header="false")
ratings.show(3)

+---+---+---+---------+
_c0|_c1|_c2| _c3|
+---+---+---+---------+
196|242| 3|881250949|
186|302| 3|891717742|
 22|377| 1|878887116|
+---+---+---+---------+
only showing top 3 rows

### Change Column names / Add header
Good functionality. Always required. Don’t forget the * in front of the list.

In [4]:
ratings = ratings.toDF(*['user_id', 'movie_id', 'rating', 'unix_timestamp'])
ratings.show(3)

+-------+--------+------+--------------+
user_id|movie_id|rating|unix_timestamp|
+-------+--------+------+--------------+
 196| 242| 3| 881250949|
 186| 302| 3| 891717742|
 22| 377| 1| 878887116|
+-------+--------+------+--------------+
only showing top 3 rows

### Some basic stats

In [6]:
print(ratings.count()) #Row Count
print(len(ratings.columns)) #Column Count

100000
4

## Different ways to present the data
* display
* show()
* toPandas()
* collect(): Whole DF to list
* take(n): Same as above but only 'n' rows

In [8]:
display(ratings.describe())

summary,user_id,movie_id,rating,unix_timestamp
count,100000,100000,100000,100000
mean,462.48475,425.53013,3.52986,8.8352885148862E8
stddev,266.61442012750905,330.79835632558473,1.1256735991443214,5343856.189502848
min,1,1,1,874724710
max,943,1682,5,893286638


In [9]:
ratings.describe().show(10)

+-------+------------------+------------------+------------------+-----------------+
summary| user_id| movie_id| rating| unix_timestamp|
+-------+------------------+------------------+------------------+-----------------+
 count| 100000| 100000| 100000| 100000|
 mean| 462.48475| 425.53013| 3.52986|8.8352885148862E8|
 stddev|266.61442012750905|330.79835632558473|1.1256735991443214|5343856.189502848|
 min| 1| 1| 1| 874724710|
 max| 943| 1682| 5| 893286638|
+-------+------------------+------------------+------------------+-----------------+

In [10]:
ratings.describe().toPandas()

,summary,user_id,movie_id,rating,unix_timestamp
0,count,100000,100000,100000,100000
1,mean,462.48475,425.53013,3.52986,8.8352885148862E8
2,stddev,266.61442012750905,330.79835632558473,1.1256735991443214,5343856.189502848
3,min,1,1,1,874724710
4,max,943,1682,5,893286638


In [11]:
cr = ratings.describe().collect()
type(cr)            # --> list

#for row in cr:
#  print(row)
  # print(type(row))     # <class 'pyspark.sql.types.Row'>
  # print(f"Mean is:{row[1]}")
  #for res in row:
  #  print(res)
  
[print(row) for row in cr]

Row(summary='count', user_id='100000', movie_id='100000', rating='100000', unix_timestamp='100000')
Row(summary='mean', user_id='462.48475', movie_id='425.53013', rating='3.52986', unix_timestamp='8.8352885148862E8')
Row(summary='stddev', user_id='266.61442012750905', movie_id='330.79835632558473', rating='1.1256735991443214', unix_timestamp='5343856.189502848')
Row(summary='min', user_id='1', movie_id='1', rating='1', unix_timestamp='874724710')
Row(summary='max', user_id='943', movie_id='1682', rating='5', unix_timestamp='893286638')
Out[11]: [None, None, None, None, None]

In [12]:
ratings.describe().take(5)

Out[12]: [Row(summary='count', user_id='100000', movie_id='100000', rating='100000', unix_timestamp='100000'),
 Row(summary='mean', user_id='462.48475', movie_id='425.53013', rating='3.52986', unix_timestamp='8.8352885148862E8'),
 Row(summary='stddev', user_id='266.61442012750905', movie_id='330.79835632558473', rating='1.1256735991443214', unix_timestamp='5343856.189502848'),
 Row(summary='min', user_id='1', movie_id='1', rating='1', unix_timestamp='874724710'),
 Row(summary='max', user_id='943', movie_id='1682', rating='5', unix_timestamp='893286638')]

In [13]:
ratings.select('user_id','movie_id').show(3)

+-------+--------+
user_id|movie_id|
+-------+--------+
 196| 242|
 186| 302|
 22| 377|
+-------+--------+
only showing top 3 rows

### Filter
Filter a dataframe using multiple conditions:

In [15]:
ratings.filter((ratings.rating==5) & (ratings.user_id==253)).show(3)

+-------+--------+------+--------------+
user_id|movie_id|rating|unix_timestamp|
+-------+--------+------+--------------+
 253| 465| 5| 891628467|
 253| 510| 5| 891628416|
 253| 183| 5| 891628341|
+-------+--------+------+--------------+
only showing top 3 rows

### Groupby
We can use groupby function with a spark dataframe too. 

Pretty much same as a pandas groupby with the exception that you will need to import pyspark.sql.functions

In [17]:
from pyspark.sql import functions as F
ratings.groupBy("user_id").agg(F.count("user_id"),F.mean("rating")).show(3)

+-------+--------------+------------------+
user_id|count(user_id)| avg(rating)|
+-------+--------------+------------------+
 148| 65| 4.0|
 463| 133|2.8646616541353382|
 471| 31|3.3870967741935485|
+-------+--------------+------------------+
only showing top 3 rows

### Sort

In [19]:
ratings.sort("user_id").show(5)

+-------+--------+------+--------------+
user_id|movie_id|rating|unix_timestamp|
+-------+--------+------+--------------+
 1| 33| 4| 878542699|
 1| 202| 5| 875072442|
 1| 160| 4| 875072547|
 1| 61| 4| 878542420|
 1| 189| 3| 888732928|
+-------+--------+------+--------------+
only showing top 5 rows

In [20]:
# descending Sort
from pyspark.sql import functions as F
ratings.sort(F.desc("user_id")).show(5)

+-------+--------+------+--------------+
user_id|movie_id|rating|unix_timestamp|
+-------+--------+------+--------------+
 943| 570| 1| 888640125|
 943| 186| 5| 888639478|
 943| 232| 4| 888639867|
 943| 58| 4| 888639118|
 943| 1067| 2| 875501756|
+-------+--------+------+--------------+
only showing top 5 rows

## Joins/Merging with Spark Dataframes

I was not able to find a pandas equivalent of merge with Spark DataFrames but we can use SQL with dataframes and thus we can merge dataframes using SQL.

Let us try to run some SQL on Ratings.

We first register the ratings df to a temporary table ratings_table on which we can run sql operations.

As you can see the result of the SQL select statement is again a Spark Dataframe.

In [22]:
ratings.registerTempTable('ratings_table')
newDF = spark.sql('select * from ratings_table where rating>4')
display(newDF)

user_id,movie_id,rating,unix_timestamp
253,465,5,891628467
286,1014,5,879781125
200,222,5,876042340
122,387,5,879270459
38,95,5,892430094
160,234,5,876861185
278,603,5,891295330
287,327,5,875333916
246,201,5,884921594
242,1137,5,879741196


In [23]:
#get one more dataframe to join
movies = spark.read.load("/FileStore/tables/u.item",format="csv", sep="|", inferSchema="true", header="false")

# change column names
movies = movies.toDF(*["movie_id","movie_title","release_date","video_release_date","IMDb_URL","unknown","Action","Adventure","Animation ","Children","Comedy","Crime","Documentary","Drama","Fantasy","Film_Noir","Horror","Musical","Mystery","Romance","Sci_Fi","Thriller","War","Western"])

display(movies)

movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western
1,Toy Story (1995),01-Jan-1995,null,http://us.imdb.com/M/title-exact?Toy%20Story%20(1995),0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,null,http://us.imdb.com/M/title-exact?GoldenEye%20(1995),0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,null,http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,null,http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995),0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,null,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
6,Shanghai Triad (Yao a yao yao dao waipo qiao) (1995),01-Jan-1995,null,http://us.imdb.com/Title?Yao+a+yao+yao+dao+waipo+qiao+(1995),0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
7,Twelve Monkeys (1995),01-Jan-1995,null,http://us.imdb.com/M/title-exact?Twelve%20Monkeys%20(1995),0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
8,Babe (1995),01-Jan-1995,null,http://us.imdb.com/M/title-exact?Babe%20(1995),0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0
9,Dead Man Walking (1995),01-Jan-1995,null,http://us.imdb.com/M/title-exact?Dead%20Man%20Walking%20(1995),0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
10,Richard III (1995),22-Jan-1996,null,http://us.imdb.com/M/title-exact?Richard%20III%20(1995),0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0


Now let us try joining the tables on movie_id to get the name of the movie in the ratings table.

There are two ways:
* Using registerTempTable(XX) + SQL
* Using SparkSQL

### 1. registerTempTable(XX) + SQL

In [25]:
# Registering movies
movies.registerTempTable('movies_table')

In [26]:
ratNmovsDF = spark.sql('''
  select 
     rt.movie_id, mv.movie_title, rt.rating
  from 
    ratings_table as rt JOIN movies_table as mv ON rt.movie_id == mv.movie_id
  ''')

display(ratNmovsDF)

movie_id,movie_title,rating
242,Kolya (1996),3
302,L.A. Confidential (1997),3
377,Heavyweights (1994),1
51,Legends of the Fall (1994),2
346,Jackie Brown (1997),1
474,Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963),4
265,"Hunt for Red October, The (1990)",2
465,"Jungle Book, The (1994)",5
451,Grease (1978),3
86,"Remains of the Day, The (1993)",3


In [27]:
# 1st
# display(ratings.alias('r').join(movies, ratings.movie_id == movies.movie_id, how='inner').select("r.movie_id","movie_title", "rating"))

# 2nd
#sel_columns = ["r.movie_id", "movie_title", "rating"]
#display(ratings.alias('r').join(movies, ratings.movie_id == movies.movie_id, how='inner').select(sel_columns))

# 3rd
sel_columns = "r.movie_id", "movie_title", "rating"
display(ratings.alias('r').join(movies, ratings.movie_id == movies.movie_id, how='inner').select(*sel_columns))

movie_id,movie_title,rating
242,Kolya (1996),3
302,L.A. Confidential (1997),3
377,Heavyweights (1994),1
51,Legends of the Fall (1994),2
346,Jackie Brown (1997),1
474,Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963),4
265,"Hunt for Red October, The (1990)",2
465,"Jungle Book, The (1994)",5
451,Grease (1978),3
86,"Remains of the Day, The (1993)",3
